## 라이브러리 설치

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 35.1 MB/s 
     |████████████████████████████████| 182 kB 72.9 MB/s 
     |████████████████████████████████| 7.6 MB 58.3 MB/s 


## 모델 호출

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("wumusill/final_project_kogpt2")
model = AutoModelForCausalLM.from_pretrained("wumusill/final_project_kogpt2")

Downloading:   0%|          | 0.00/301 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.25M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/123 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/513M [00:00<?, ?B/s]

## 삼행시 함수

In [3]:
import torch

# GPU 사용 여부
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")

model.to(device)
model.eval()
print(device)

cuda:0


In [2]:
def mind(input_letter):
    # 두음 법칙 사전
    dooeum = {"라":"나", "락":"낙", "란":"난", "랄":"날", "람":"남", "랍":"납", "랑":"낭", 
          "래":"내", "랭":"냉", "냑":"약", "략":"약", "냥":"양", "량":"양", "녀":"여", 
          "려":"여", "녁":"역", "력":"역", "년":"연", "련":"연", "녈":"열", "렬":"열", 
          "념":"염", "렴":"염", "렵":"엽", "녕":"영", "령":"영", "녜":"예", "례":"예", 
          "로":"노", "록":"녹", "론":"논", "롱":"농", "뢰":"뇌", "뇨":"요", "료":"요", 
          "룡":"용", "루":"누", "뉴":"유", "류":"유", "뉵":"육", "륙":"육", "륜":"윤", 
          "률":"율", "륭":"융", "륵":"늑", "름":"늠", "릉":"능", "니":"이", "리":"이", 
          "린":'인', '림':'임', '립':'입'}
    # 결과물을 담을 list
    res_l = []

    # 한 글자씩 인덱스와 함께 가져옴
    for idx, val in enumerate(input_letter):
        # 두음 법칙 적용
        if val in dooeum.keys():
            val = dooeum[val]

        # 만약 idx 가 0 이라면 == 첫 글자
        if idx == 0:
            # 첫 글자 인코딩
            input_ids = tokenizer.encode(
            val, add_special_tokens=False, return_tensors="pt")
            
            # 첫 글자 인코딩 값으로 문장 생성
            output_sequence = model.generate(
                input_ids=input_ids.to(device), 
                do_sample=True,
                max_length=42,
                min_length=5,
                temperature=0.9, 
                repetition_penalty=1.5,
                no_repeat_ngram_size=2
                )
        
        # 첫 글자가 아니라면
        else:
            # 좀더 매끄러운 삼행시를 위해 이전 문장이랑 현재 음절 연결
            # 이후 generate 된 문장에서 이전 문장에 대한 데이터 제거
            link_with_pre_sentence = " ".join(res_l) + " " + val  
            # print(link_with_pre_sentence)

            # 연결된 문장을 인코딩
            input_ids = tokenizer.encode(
            link_with_pre_sentence, add_special_tokens=False, return_tensors="pt")

            # 인코딩 값으로 문장 생성
            output_sequence = model.generate(
                input_ids=input_ids.to(device), 
                do_sample=True,
                max_length=42,
                min_length=len_sequence + 2,
                temperature=0.9,
                repetition_penalty=1.5,
                no_repeat_ngram_size=2)

        # 생성된 문장 리스트로 변환 (인코딩 되어있고, 생성된 문장 뒤로 padding 이 있는 상태)
        generated_sequence = output_sequence.tolist()[0]

        # padding index 앞까지 slicing 함으로써 padding 제거, padding이 없을 수도 있기 때문에 조건문 확인 후 제거
        if tokenizer.pad_token_id in generated_sequence:
            generated_sequence = generated_sequence[:generated_sequence.index(tokenizer.pad_token_id)]
        
        # 첫 글자가 아니라면, generate 된 음절만 결과물 list에 들어갈 수 있게 앞 문장에 대한 인코딩 값 제거
        # print(generated_sequence)
        if idx != 0:
            # 이전 문장의 길이 이후로 슬라이싱해서 앞 문장 제거
            generated_sequence = generated_sequence[len_sequence:]

            # 다음 음절을 위해 길이 갱신
            len_sequence += len(generated_sequence)        
        
        # 첫 글자라면
        else:
            # 시퀀스 길이 저장
            len_sequence = len(generated_sequence)
        
        # print(last_sequence)

        # 결과물 디코딩
        decoded_sequence = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)

        # 결과물 리스트에 담기
        res_l.append(decoded_sequence)

        # print(res_l)

    # 결과물 list에서 한 줄씩 출력
    for letter, res in zip(input_letter, res_l):
        print(f"{letter} :", res)

In [39]:
mind("권소윤")

권 : 권태기가 아냐 이건 끝 동백기 전에
소 : 소릴 질러봐 가끔씩 네가 날 향해
윤 : 윤 내맘을 보여줄래


In [40]:
mind("코코아")

코 : 코 끝을 스쳐간 낯익은 향기에
코 : 코끝이 시려
아 : 아련한 그리움


In [41]:
mind("해파리")

해 : 해줄수 없는 내가 미워요
파 : 파 묻힌 내 두눈이
리 : 이젠여 안녕


In [42]:
mind("아이스크림")

아 : 아련함에 시린 마음도
이 : 이젠
스 : 스산한 걸
크 : 크진 않아도
림 : 임오되는라면 넘치도록 사랑했던 우리 얘기야


In [43]:
mind("파이널")

파 : 파랗게 물든 푸르른 하늘
이 : 이젠
널 : 널 만나질 테니까


In [44]:
mind("문종현")

문 : 문득 스친 네 얼굴 생각나 눈물이
종 : 종일 나
현 : 현빈


In [45]:
mind("이정은")

이 : 이 마음속에 그대 모습
정 : 정녕 떠나갔나
은 : 은빛 간직하고 있네


In [46]:
mind("권소희")

권 : 권태로 변해 두 눈을 가려도
소 : 소용이 없어
희 : 희미한 빛조차


In [ ]:
mind("박건영")

박 : 박혀있다
건 : 건네주던 니가
영 : 영원을 약속했던


In [ ]:
mind("정재영")

정 : 정말로 그건 거짓말이야
재 : 재밌는 일이야
영 : 영영


In [ ]:
n_line_poem("구자현")

구 : 구겨 구겨져 버린 편지를 손에 쥐고
자 : 자꾸 물었어
현 : 현관문을 열었어 굳이 버렸어


In [ ]:
n_line_poem("김의준")

김 : 김밥 만두 라면 나만 알 수 있겠죠 그대가 숨겨 논 환상은
의 : 의 헤어짐의 새 이름
준 : 준 오직 나만 느낄 수 있죠 모든 순간을 그대가


In [ ]:
n_line_poem("최지영")

최 : 최선을 다한 그 쌓인 눈들도
지 : 지워볼게
영 : 영영 피어볼게 난 너의 정든 곁을 지켜줄게


In [ ]:
dooeum = {"라":"나", "락":"낙", "란":"난", "랄":"날", "람":"남", "랍":"납", "랑":"낭", 
          "래":"내", "랭":"냉", "냑":"약", "략":"약", "냥":"양", "량":"양", "녀":"여", 
          "려":"여", "녁":"역", "력":"역", "년":"연", "련":"연", "녈":"열", "렬":"열", 
          "념":"염", "렴":"염", "렵":"엽", "녕":"영", "령":"영", "녜":"예", "례":"예", 
          "로":"노", "록":"녹", "론":"논", "롱":"농", "뢰":"뇌", "뇨":"요", "료":"요", 
          "룡":"용", "루":"누", "뉴":"유", "류":"유", "뉵":"육", "륙":"육", "륜":"윤", 
          "률":"율", "륭":"융", "륵":"늑", "름":"늠", "릉":"능", "니":"이", "리":"이", 
          "린":'인', '림':'임', '립':'입'}